In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wherever/sample_submission.csv
/kaggle/input/song-popularity/Kaggle competiton/train.csv
/kaggle/input/song-popularity/Kaggle competiton/test.csv


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('ggplot')


data_path = "../input/song-popularity/Kaggle competiton/"#"/kaggle/input/song-popularity/Kaggle competition/"
train_df = pd.read_csv(data_path+'train.csv')
train_df = train_df.astype({'key': 'category', 'audio_mode': 'category', 'time_signature':'category'})

test_df = pd.read_csv(data_path+'test.csv')
test_df = test_df.astype({'key': 'category', 'audio_mode': 'category','time_signature':'category'})

In [3]:
train_df.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,NaN,-5.619088,0,0.082570,158.386236,4,0.734642,0
1,1,NaN,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1,0.127358,102.752988,3,0.711531,1
2,2,193213.0,NaN,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0,0.052282,178.685791,3,0.425536,0
3,3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0,0.035618,128.715630,3,0.453597,0
4,4,165969.0,0.493017,NaN,0.740982,0.002033,10.0,0.094891,-2.684095,0,0.050746,121.928157,4,0.741311,0


In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer()#random_state=0, max_iter=100, initial_strategy='mean'


In [5]:
train_im = pd.DataFrame(imputer.fit_transform(train_df.loc[:,train_df.columns!='song_popularity']))
test_im = pd.DataFrame(imputer.transform(test_df))

train_im['song_popularity'] = train_df['song_popularity']

In [6]:
train_im.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,song_popularity
0,0.0,212990.000000,0.642286,0.856520,0.707073,0.002001,10.0,0.216704,-5.619088,0.0,0.082570,158.386236,4.0,0.734642,0
1,1.0,197397.101336,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1.0,0.127358,102.752988,3.0,0.711531,1
2,2.0,193213.000000,0.160052,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0.0,0.052282,178.685791,3.0,0.425536,0
3,3.0,249893.000000,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0.0,0.035618,128.715630,3.0,0.453597,0
4,4.0,165969.000000,0.493017,0.645651,0.740982,0.002033,10.0,0.094891,-2.684095,0.0,0.050746,121.928157,4.0,0.741311,0


In [7]:
train_im.columns = train_df.columns
test_im.columns = test_df.columns
working_dir = "/kaggle/working/"
train_df.to_csv(working_dir+'train_impu.csv', index=False)
test_df.to_csv(working_dir+'test_impu.csv', index=False)

In [8]:
# path to training data

train_filename = '/kaggle/working/train_impu.csv'
# path to output folder to store artifacts
output =  '/kaggle/working/random_forest'

test_filename ='/kaggle/working/test_impu.csv'


In [9]:
print(*range(2,51,3))

2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 50


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, make_scorer

# Create the param grid
n_estimators = [20*i for i in range(6,15)]
max_samples =  [1000*i for i in range(1,5)]
max_features = ['auto', 'sqrt']
max_depth = range(4,15)
min_samples_split = range(11,51,5)
min_samples_leaf = range(11,51,5)
bootstrap = [True]
criterion = ['gini','entropy']
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_samples': max_samples,
               'criterion':criterion,
               'bootstrap':bootstrap
             }


rf_classifier = RandomForestClassifier()
#rf_grid = GridSearchCV(estimator = rf_classifier, param_grid = param_grid, cv = 5, verbose=2, n_jobs = -1)
rf_RandomGrid = RandomizedSearchCV(estimator = rf_classifier, param_distributions = param_grid, cv = 5, verbose=2, n_jobs = 4, n_iter = 1000, scoring=make_scorer(roc_auc_score))


In [11]:
#help(RandomizedSearchCV)

In [ ]:
X = train_im.loc[:,train_im.columns!='song_popularity']
y = train_im['song_popularity']
                             
rf_RandomGrid.fit(X,y)

In [ ]:
rf_RandomGrid.best_params_

In [ ]:
print (f'Train Accuracy - : {rf_RandomGrid.score(X,y):.3f}')


In [ ]:
roc_auc_score(y,rf_RandomGrid.predict_proba(X)[:,1])

In [ ]:
test_pred = rf_RandomGrid.predict_proba(test_im)
test = pd.read_csv("../input/wherever/sample_submission.csv")
test['song_popularity'] = test_pred[:,1]
test.to_csv("submission.csv", index=False)

In [ ]:
test.head()